In [3]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [29]:
# PASSO 1: INSTALAR BIBLIOTECAS (Execute em uma célula separada no Colab, se necessário)
# !pip install -q google-adk google-generativeai

#---------------------------------------------------------------------
# PASSO 2: CONFIGURAÇÕES INICIAIS (Chave API e importações essenciais)
#---------------------------------------------------------------------
import os
import sys
import google.generativeai as genai_sdk_module # Usado para genai.configure()
# IPython.display é para ambientes Jupyter/Colab
IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    from IPython.display import display, Markdown
    # from google.colab import files # REMOVIDO - não vamos mais fazer download de arquivo
else:
    def display(obj):
        if hasattr(obj, 'data') and 'text/markdown' in obj.data: print(obj.data['text/markdown'])
        else: print(obj)
    def Markdown(text):
        class MarkdownObject:
            def __init__(self, data): self.data = {'text/markdown': data}
        return MarkdownObject(text)

# Configuração da API Key
GOOGLE_API_KEY = None
try:
    if IS_COLAB:
        from google.colab import userdata
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY: GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if GOOGLE_API_KEY:
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        genai_sdk_module.configure(api_key=GOOGLE_API_KEY)
    else:
        print("ERRO CRÍTICO: GOOGLE_API_KEY não encontrada. Configure-a.")
        exit()
except ImportError:
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if GOOGLE_API_KEY: genai_sdk_module.configure(api_key=GOOGLE_API_KEY)
    else: print("ERRO CRÍTICO: GOOGLE_API_KEY não encontrada como variável de ambiente."); exit()
except Exception as e:
    print(f"ERRO CRÍTICO ao configurar a API Key: {e}"); exit()

#---------------------------------------------------------------------
# PASSO 3: RESTANTE DAS IMPORTAÇÕES E CÓDIGO DO CHATBOT
#---------------------------------------------------------------------
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.genai import types # Usado pelo ADK
import datetime
import json
import re

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# --- Funções Auxiliares e de Agente ---
def call_agent(
    agent: Agent, message_text: str, session_service: InMemorySessionService,
    user_id: str, session_id_for_conversation: str
) -> str:
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id=user_id, session_id=session_id_for_conversation, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response.strip()

def simular_investimento_juros_compostos(
    valor_inicial, aporte_mensal, taxa_juros_anual_nominal, periodo_anos, taxa_inflacao_anual
):
    try:
        valor_inicial = float(valor_inicial); aporte_mensal = float(aporte_mensal)
        taxa_juros_anual_nominal = float(taxa_juros_anual_nominal)
        periodo_anos = float(periodo_anos); taxa_inflacao_anual = float(taxa_inflacao_anual)
    except ValueError: raise ValueError("Todos os parâmetros da simulação devem ser números.")
    if periodo_anos <= 0: raise ValueError("O período em anos deve ser positivo para a simulação.")
    if taxa_juros_anual_nominal < 0: taxa_juros_anual_nominal = 0.0

    taxa_juros_nominal_mensal = (1 + taxa_juros_anual_nominal)**(1/12) - 1
    taxa_inflacao_mensal = (1 + taxa_inflacao_anual)**(1/12) - 1

    taxa_juros_real_mensal = 0
    if (1 + taxa_inflacao_mensal) != 0:
        taxa_juros_real_mensal = ((1 + taxa_juros_nominal_mensal) / (1 + taxa_inflacao_mensal)) - 1
    else: taxa_juros_real_mensal = taxa_juros_nominal_mensal

    num_meses = int(round(periodo_anos * 12));
    if num_meses == 0 and periodo_anos > 0: num_meses = 1

    saldo_real = valor_inicial
    valor_total_aportado_nominal = valor_inicial

    for i in range(num_meses):
        saldo_real += aporte_mensal
        if i == 0 and valor_inicial > 0 and aporte_mensal > 0 :
             pass
        elif aporte_mensal > 0 :
             valor_total_aportado_nominal += aporte_mensal
        saldo_real *= (1 + taxa_juros_real_mensal)

    if aporte_mensal == 0 : valor_total_aportado_nominal = valor_inicial
    elif num_meses > 0 : valor_total_aportado_nominal = valor_inicial + (aporte_mensal * num_meses)

    return {
        "valor_final_real": round(saldo_real, 2),
        "total_investido_nominal": round(valor_total_aportado_nominal, 2)
    }

# Função exibir_grafico_investimento FOI REMOVIDA

def agente_consultor_financeiro():
    return Agent(
        name="ConsultorFinanceiro", model="gemini-2.5-pro-preview-03-25",
        instruction="""Você é o ConsultorFinanceiro, especialista em produtos/serviços financeiros, mercado e educação financeira. Sua função é informar, comparar produtos, explicar detalhes técnicos, analisar o mercado, educar sobre conceitos, finanças pessoais, orçamento, economia e estratégias para sair de dívidas. Mantenha o contexto. Use linguagem acessível. **NÃO FAÇA SIMULAÇÕES DE INVESTIMENTO DIRETAMENTE.** Se o usuário pedir uma simulação de investimento, explique que temos o `SimuladorFinanceiro` para isso e pergunte se ele deseja ser direcionado: "Nosso `SimuladorFinanceiro` pode te ajudar com isso. Gostaria de fazer uma simulação agora?". Se ele confirmar, o sistema o direcionará. Para outros tópicos, prossiga.

        Responsabilidades:
        1. PRODUTOS E MERCADO: Detalhes de contas, poupança (rendimento atual vs Selic), investimentos (CDB, LCI, Tesouro, Ações - características, como investir, riscos, tributação, performance se `google_search` achar), empréstimos (taxas médias, CET, condições), seguros, cartões, consórcios, previdência. Análise de indicadores (Selic, IPCA, CDI) e seus impactos.
        2. EDUCAÇÃO FINANCEIRA: Conceitos (inflação, Selic, juros, orçamento, poupança, CDI, PIX, CET, tipos de investimento), dicas (orçamento 50/30/20, economizar, metas, reserva de emergência), planejamento e dívidas (estratégias gerais para sair de dívidas - conceitual), glossário, riscos (educativo).

        Instruções Importantes: Use `google_search` para dados atuais (taxas, rendimentos) e informações de fontes confiáveis (BCB, CVM, Anbima, portais de notícias, sites de educação financeira). Seja claro, preciso, atual, objetivo em comparações, e neutro. Enfatize riscos. DISCLAIMER OBRIGATÓRIO ao falar de investimentos/crédito ou dar dicas que possam ser interpretadas como conselho: "Lembre-se que estas são informações gerais e educativas, não constituindo recomendação ou aconselhamento financeiro personalizado. Para decisões específicas, consulte um profissional qualificado." Não peça dados pessoais.
        """, tools=[google_search])

def agente_atendimento_cliente():
    return Agent(
        name="SuporteOperacional", model="gemini-2.0-flash-001",
        instruction="""Você é o SuporteOperacional, focado em processos e dúvidas operacionais. Mantenha o contexto. Se a dúvida do usuário for sobre conceitos financeiros, produtos de investimento ou simulações, o ConsultorFinanceiro ou o SimuladorFinanceiro são mais apropriados. Responsabilidades: Processos de Contas, Cartões, Transações (como fazer), Suporte Técnico Básico (acesso app/site, erros), Documentos (informe IR), Canais de Atendimento. Instruções: (1)FONTES (2)CLAREZA (3)SEM DADOS PESSOAIS (4)NÃO EXECUTE TRANSAÇÕES (5)ATUALIDADE DE CONTATOS.""",
        tools=[google_search])

def agente_simulador_financeiro():
    return Agent(
        name="SimuladorFinanceiro", model="gemini-2.0-flash-001",
        instruction="""
        Você é o SimuladorFinanceiro. Sua ÚNICA TAREFA é interagir com o usuário para coletar CINCO parâmetros para uma simulação de investimento com juros compostos (resultado ajustado pela inflação):
        1.  'valor_inicial': O montante que o usuário já tem para começar (ex: 1000.00). Se não informado, pergunte. Pode ser 0.
        2.  'aporte_mensal': O valor que o usuário pretende adicionar todo mês (ex: 100.00). Se não informado, pergunte. Pode ser 0.
        3.  'taxa_juros_anual_nominal': A taxa de juros NOMINAL ANUAL esperada, em formato DECIMAL (ex: para 10% ao ano, use 0.10).
            - Se o usuário fornecer um valor numérico (ex: "10% a.a.", "0.12 ao ano"), use esse valor após convertê-lo para decimal.
            - Se o usuário pedir para usar a 'taxa Selic atual' ou 'taxa CDI atual', use a ferramenta `google_search` para buscar o valor mais recente (ex: 'taxa Selic hoje Brasil', 'taxa CDI hoje Brasil'). APRESENTE o valor encontrado (e a data, se possível) e PERGUNTE SE O USUÁRIO CONFIRMA o uso dessa taxa. Ex: "A Selic atual está em X% ao ano. Podemos usar esta taxa para a simulação?". Somente após a confirmação, use este valor convertido para decimal.
            - Se o usuário fornecer uma taxa mensal (ex: "1% ao mês"), CONVERTA para anual usando (1 + taxa_mensal)^12 - 1, mostre o cálculo e a taxa anual convertida, e CONFIRME com o usuário se ele aceita usar essa taxa anual.
            - Se nenhuma taxa específica for fornecida ou solicitada, pergunte: "Qual a taxa de juros nominal anual que devemos considerar para esta simulação (ex: 8% ao ano)?"
        4.  'periodo_anos': O tempo total em ANOS (ex: 5.0). Pode ser decimal. Se o usuário der em meses, CONVERTA (meses / 12) e confirme o valor em anos. Se não informado, pergunte.
        5.  'taxa_inflacao_anual_esperada': A inflação ANUAL esperada, em formato DECIMAL (ex: 0.04 para 4%).
            - Se o usuário fornecer, use esse valor (convertendo % para decimal).
            - Se não fornecer, PERGUNTE: "Qual a taxa de inflação anual que devemos projetar (ex: 3.5% ao ano)? Se preferir, posso buscar uma projeção recente para usarmos."
            - Se o usuário pedir para você buscar ou aceitar sua sugestão de busca, use `google_search` (ex: 'projeção inflação Brasil anual IPCA'). Apresente o valor encontrado (e a data, se possível) e PERGUNTE SE O USUÁRIO CONFIRMA o uso dessa taxa.

        Seja educado e direto. Peça UM PARÂMETRO POR VEZ se necessário.
        Confirme TODOS os cinco valores com o usuário antes de finalizar. Ex: "Ok, então os dados para a simulação são: Valor Inicial R$1000, Aporte Mensal R$100, Taxa de Juros Nominal Anual 10% (0.10), Período 5 anos, e Inflação Anual Esperada 4% (0.04). Correto?"

        Quando tiver TODOS OS CINCO PARÂMETROS NUMÉRICOS E CONFIRMADOS PELO USUÁRIO:
        Responda EXCLUSIVAMENTE com a string de ação no seguinte formato:
        "ACAO_SIMULAR:{{'valor_inicial': <float>, 'aporte_mensal': <float>, 'taxa_juros_anual_nominal': <float_decimal>, 'periodo_anos': <float>, 'taxa_inflacao_anual_esperada': <float_decimal>}}"

        Exemplo de sua RESPOSTA FINAL (depois de todas as coletas e confirmações):
        "Entendido! Com os dados confirmados: valor inicial R$1000.00, aporte mensal R$500.00, taxa de juros nominal anual 0.1405 (14.05%), período de 1.33 anos, e inflação anual esperada 0.03 (3%).
        ACAO_SIMULAR:{{'valor_inicial': 1000.00, 'aporte_mensal': 500.00, 'taxa_juros_anual_nominal': 0.1405, 'periodo_anos': 1.33, 'taxa_inflacao_anual_esperada': 0.03}}"

        Não adicione mais nada à string ACAO_SIMULAR. Se a pergunta não for para coletar dados para simulação, diga: "Minha especialidade é coletar dados para simulações. O ConsultorFinanceiro pode ajudar com outras dúvidas."
        """,
        tools=[google_search]
    )

# (MANTENHA TODAS AS SUAS IMPORTAÇÕES E DEFINIÇÕES DE FUNÇÃO ANTERIORES AQUI:
# IS_COLAB, display, Markdown, Configuração da API Key,
# call_agent, simular_investimento_juros_compostos,
# agente_consultor_financeiro, agente_atendimento_cliente, agente_simulador_financeiro,
# e adicionar_variacoes_sem_acento)

# --- INÍCIO DA FUNÇÃO CHATBOT_FINANCEIRO CORRIGIDA ---
def chatbot_financeiro():
    consultor = agente_consultor_financeiro()
    suporte = agente_atendimento_cliente()
    simulador = agente_simulador_financeiro()

    session_service = InMemorySessionService()
    user_id_conversa = "user_chat_01"
    session_id_consultor = f"conversa_com_{consultor.name}_para_{user_id_conversa}"
    session_id_suporte = f"conversa_com_{suporte.name}_para_{user_id_conversa}"
    session_id_simulador = f"conversa_com_{simulador.name}_para_{user_id_conversa}"

    try:
        session_service.create_session(app_name=consultor.name, user_id=user_id_conversa, session_id=session_id_consultor)
        session_service.create_session(app_name=suporte.name, user_id=user_id_conversa, session_id=session_id_suporte)
        session_service.create_session(app_name=simulador.name, user_id=user_id_conversa, session_id=session_id_simulador)
    except Exception as e:
        print(f"Aviso: Erro ao tentar criar sessões iniciais: {e}")

    print("🏦 Bem-vindo ao Chatbot Financeiro Completo! 🏦")
    print("Sou seu assistente virtual para o mundo financeiro. Como posso te ajudar hoje?")
    print(f"Posso te ajudar com nosso {consultor.name} (para informações e educação financeira), nosso {simulador.name} (para simulações de investimento), ou nosso {suporte.name} (para processos e suporte técnico).")
    print("\nIMPORTANTE: Eu forneço informações gerais e educativas, e não aconselhamento financeiro personalizado ou recomendações específicas.")
    print(f"As condições de mercado podem mudar rapidamente. Para decisões financeiras importantes, especialmente investimentos, consulte um profissional qualificado. (Data de referência: {datetime.date.today().strftime('%d/%m/%Y')})\n")

    last_agent_used_name = None
    last_user_question = ""
    last_agent_response_summary = ""
    pending_redirection_target_agent_name = None
    last_simulation_results_context = None

    saudacoes_comuns = ["oi", "olá", "ola", "bom dia", "boa tarde", "boa noite", "e aí", "e ai", "salve", "opa", "iai", "blz", "beleza"]

    def adicionar_variacoes_sem_acento(lista_keywords):
        novas_keywords = set(lista_keywords)
        for kw_item in lista_keywords:
            if not isinstance(kw_item, str): continue
            kw_sem_acento = kw_item.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')\
                              .replace('â', 'a').replace('ê', 'e').replace('ô', 'o')\
                              .replace('ã', 'a').replace('õ', 'o').replace('ç', 'c')
            if kw_sem_acento != kw_item: novas_keywords.add(kw_sem_acento)
        return list(novas_keywords)

    frases_de_encerramento_base = [
        "sair", "finalizar", "encerrar", "terminar", "exit", "quit",
        "tchau", "tchau tchau", "tchao", "tcha", "xau", "chau", "tial",
        "ate logo", "até logo", "ate mais", "até mais", "ate", "até",
        "falou", "flw", "fui", "adeus", "vlw",
        "é só isso", "por enquanto é só", "era isso",
        "só isso mesmo", "já tirei minhas dúvidas",
        "sem mais perguntas", "não preciso de mais nada",
        "nada mais", "nenhuma outra pergunta",
        "obrigado, é só", "valeu, é só",
        "obrigado é só", "valeu é só",
        "grato, é só",
        "ok, obrigado", "obrigado", "valeu",
        "tudo certo, obrigado",
        "ok, é só",
        "chega", "basta", "já deu", "por hoje é só",
        "vou embora", "estou de saida", "to de saida", "preciso ir", "tenho que ir", "hora de ir", "ja vou",
        "por mim chega", "ja deu pra mim", "fechou", "encerro por aqui", "cansei",
        "abracos", "abraço", "abs", "beijo", "beijos", "bjs",
        "fique bem", "fica bem", "se cuida",
        "obrigado, vou nessa", "valeu, to indo", "agradecido, vou partir"
    ]
    frases_de_encerramento = adicionar_variacoes_sem_acento(frases_de_encerramento_base)

    affirmatives_for_redirection = ["sim", "s", "quero", "pode ser", "claro", "com certeza", "gostaria", "prosseguir", "aceito", "ok", "sim, por favor", "positivo", "bora", "manda bala", "sim quero", "sim gostaria", "perfeito"]

    # Lista de follow-up genérico COMPLETA
    keywords_follow_up_generico = adicionar_variacoes_sem_acento([
        "e sobre", "mais sobre", "continue", "e aquilo", "sim", "não", "ok", "entendi",
        "explique melhor sobre isso", "detalhe mais isso", "fale mais sobre isso",
        "desenvolva isso", "conte me mais", "me fale mais", "como assim", "por que", "pq",
        "por quê", "como é", "me explica de novo"
    ])

    keywords_pedir_consultor = adicionar_variacoes_sem_acento(["falar com consultor", "quero o consultor", "consultor financeiro pode", "o consultor pode", "consultor", "analista", "especialista financeiro", "educador financeiro"])
    keywords_pedir_suporte = adicionar_variacoes_sem_acento(["falar com suporte", "quero o suporte", "suporte operacional pode", "o suporte pode", "suporte", "atendimento", "ajuda técnica"])
    keywords_pedir_simulador = adicionar_variacoes_sem_acento(["falar com simulador", "quero o simulador", "simulador financeiro pode", "o simulador pode", "simulador", "fazer simulação", "simula para mim", "simular meu investimento", "simular", "simulacao"])

    # --- KEYWORDS COMPLETAS E ATUALIZADAS ---
    keywords_consultor_forte = adicionar_variacoes_sem_acento([
        "rentabilidade de", "qual melhor investimento para", "onde investir meu dinheiro em", "risco de investir em", "perfil de investidor",
        "comparar cdb com", "comparar fundos de", "cdb ou lci qual o melhor", "qual rende mais cdb ou", "comparativo de fundos de",
        "análise da ação", "mercado de ações hoje", "cotação do dólar hoje", "previsão da inflação para", "ipca acumulado", "selic e investimentos qual impacto",
        "tesouro selic", "taxa lci", "melhor cdb", "etf rentabilidade", "fundo imobiliário", "fiis",
        "carteira de investimentos para", "alocação de ativos para", "diversificação de carteira para", "diversificar investimentos",
        "tributação de cdb", "imposto de renda sobre ações", "melhor previdência privada",
        "empréstimo para quitar dívidas", "consolidar dívidas",
        "como economizar dinheiro", "dicas para guardar dinheiro", "organizar meu dinheiro", "fazer um orçamento pessoal",
        "planejamento financeiro pessoal", "como sair das dívidas", "minhas dívidas", "ajuda com dívidas",
        "eliminar dívidas", "quitar dívidas", "negociar dívidas", "estou endividado",
        "reserva de emergência", "educação financeira", "aprender sobre finanças", "curso de finanças",
        "o que é inflação", "o que é taxa selic", "o que é cdi", "explicar juros compostos",
        "glossário financeiro", "conceitos de investimento", "como controlar gastos",
        "o que é poupança", "como funciona a poupança", "o que é um cdb", "o que é lci",
        "o que é um fundo de investimento", "o que são ações", "o que é tesouro direto", "o que é diversificação",
        "método 50/30/20", "como definir metas financeiras", "planejar aposentadoria"
    ])
    keywords_consultor_geral = adicionar_variacoes_sem_acento([
        "investimento", "investir", "juros", "taxa de juros", "taxas", "financiamento", "empréstimo", "crédito",
        "seguro", "previdência", "consórcio", "ações", "fundos", "poupança", "dinheiro", "finanças pessoais",
        "economia", "mercado financeiro", "aplicação financeira", "lucro", "perda", "risco",
        "corretora", "banco de investimento", "consultoria financeira",
        "dicas de investimento", "dicas de economia", "organização financeira", "dívida", "divida", "dívidas", "dividas", "orçamento", "planejamento"
    ])
    # Lista de follow-up do consultor COMPLETA
    keywords_consultor_follow_up = adicionar_variacoes_sem_acento([
        "resumir isso", "resuma sobre", "explique melhor esse produto", "explique melhor esse conceito",
        "detalhe mais sobre", "qual o risco específico de", "desvantagens de", "vantagens de",
        "e as taxas", "quais as condições", "quais as condicoes", "como funciona o rendimento",
        "qual o imposto", "como declarar",
        "me ensina mais", "quero aprender mais", "tem mais dicas sobre", "como faço isso na prática",
        "me dê um exemplo prático", "qual a diferença entre", "explique de novo"
    ])
    keywords_simulador_forte = adicionar_variacoes_sem_acento([
        "simular investimento", "simulação de investimento", "calcular investimento", "projetar investimento", "simulador de investimento",
        "quanto rende se eu investir", "simulador de juros compostos", "simula pra mim", "faz uma simulação", "simular aposentadoria",
        "simular valor inicial", "simular aporte mensal", "simular com taxa", "simular por x anos", "simular por alguns anos", "simule",
        "quero uma simulação", "gostaria de simular", "pode simular"
    ])
    keywords_simulador_geral = adicionar_variacoes_sem_acento(["simulação", "simulacao", "simulador", "projeção", "projecao", "calcular juros", "rendimento futuro", "simular", "projeta"])

    keywords_suporte_forte = adicionar_variacoes_sem_acento([
        "abrir conta", "encerrar conta", "cancelar conta", "excluir conta", "deletar conta", "como excluir conta", "como encerrar conta", "como cancelar minha conta",
        "problema com minha conta", "conta bloqueada", "saldo incorreto", "extrato errado",
        "não consigo acessar", "esqueci minha senha", "mudar senha", "senha bloqueada", "app não funciona", "aplicativo não funciona",
        "internet banking erro", "site não funciona", "portal com problema", "fatura do cartão", "limite do cartão", "aumentar limite",
        "contestar compra", "compra não reconhecida", "estorno", "segunda via do cartão", "bloquear cartão", "desbloquear cartão",
        "cartão clonado", "cartão roubado", "perdi meu cartão", "cartão não passa", "aproximação não funciona", "pagamento recusado com cartão",
        "portabilidade de salário", "comprovante de rendimentos", "informe de rendimentos", "declaração de imposto de renda",
        "internet não funciona", "minha internet não funciona", "sem internet", "sem conexão", "não consigo conectar", "falha na conexão",
        "problema de acesso", "erro no login", "login não funciona", "site fora do ar", "site com problema", "aplicativo com problema",
        "app com problema", "falha técnica", "suporte técnico", "problema técnico", "erro no sistema", "sistema indisponível",
        "não carrega", "página não abre", "não funciona", "está fora do ar", "código de erro", "mensagem de erro", "sistema lento", "lentidão",
        "fraude", "golpe", "phishing", "segurança da conta", "proteger minha conta", "token", "autenticação de dois fatores", "validar identidade",
        "transferência não chegou", "pix devolvido", "problema no boleto", "pagamento duplicado", "débito indevido", "cobrança indevida"
    ])
    keywords_suporte_geral = adicionar_variacoes_sem_acento([
        "cartão", "conta corrente", "conta poupança", "conta salario", "conta digital", "documento", "boleto", "transferência", "pix", "pagamento", "extrato", "saldo",
        "telefone", "contato", "agência", "falar com atendente", "aplicativo", "login", "acesso", "senha", "cadastro", "atualizar cadastro",
        "meus dados", "horário de atendimento", "fale conosco", "central de atendimento", "chat", "dúvida operacional", "ajuda com app", "ajuda com site",
        "serviços bancários", "operações bancárias", "configurações da conta", "limites transacionais", "agendamento", "itau", "bradesco", "santander", "banco do brasil", "caixa"
    ])
    keywords_sobre_simulacao_anterior = adicionar_variacoes_sem_acento([
        "rendimento", "juros", "lucro", "ganho", "resultado", "diferença", "quanto rendeu", "qual foi o juros", "qual foi o lucro", "qual foi o ganho", "qual a diferenca",
        "resultado da simulação", "juros da simulação", "lucro da simulação", "ganho da simulação", "diferença da simulação", "valor final simulado"
    ])
    # --- FIM DAS KEYWORDS ---

    while True:
        pergunta_original_usuario = input("❓ Digite sua pergunta: ").strip()
        if not pergunta_original_usuario: continue
        pergunta_lower = pergunta_original_usuario.lower()

        if pergunta_lower in frases_de_encerramento:
            print("🤖 Entendido. Espero ter ajudado! Se tiver mais dúvidas no futuro, é só chamar. Até a próxima!")
            pending_redirection_target_agent_name = None; last_simulation_results_context = None
            break

        if pergunta_lower in saudacoes_comuns:
            print(f"🤖 {pergunta_original_usuario.capitalize()}! Sou seu assistente financeiro virtual.")
            print(f"   Posso te ajudar com nosso {consultor.name} (para informações e educação financeira), nosso {simulador.name} (para simulações de investimento), ou nosso {suporte.name} (para processos e suporte técnico). Como posso ajudar hoje?")
            last_agent_used_name = None; last_user_question = ""; last_agent_response_summary = ""; pending_redirection_target_agent_name = None; last_simulation_results_context = None
            print("-" * 50); continue

        agente_escolhido = None
        session_id_para_agente_atual = None
        pergunta_para_agente = pergunta_original_usuario

        # PRIORIDADE 0: Responder a perguntas sobre a última simulação feita
        if last_simulation_results_context and any(kw in pergunta_lower for kw in keywords_sobre_simulacao_anterior):
            try:
                vfr = last_simulation_results_context['valor_final_real']
                tin = last_simulation_results_context['total_investido_nominal']
                ganho_real_calculado = vfr - tin
                display(Markdown(f"🤖 **Analisando os resultados da sua última simulação:**"))
                display(Markdown(f"- Montante final (em poder de compra de hoje): **R$ {vfr:,.2f}**"))
                display(Markdown(f"- Total nominal investido (valor inicial + todos os aportes): **R$ {tin:,.2f}**"))
                display(Markdown(f"- Seu Lucro/Rendimento Real (diferença, já descontada a inflação) foi de: **R$ {ganho_real_calculado:,.2f}**"))
                print("   Lembre-se que este valor representa o ganho em poder de compra de hoje, após descontar a inflação utilizada na simulação.")
                last_user_question = pergunta_original_usuario
                last_agent_response_summary = f"Calculado lucro/rendimento real da última simulação: R$ {ganho_real_calculado:,.2f}"
                print("-" * 50); continue
            except KeyError: print(f"🤖 Desculpe, não encontrei dados da simulação anterior."); last_simulation_results_context = None
            except Exception as e: print(f"🤖 Desculpe, erro ao calcular rendimento da simulação: {e}")

        # --- LÓGICA DE ROTEAMENTO ---
        # (Coloque aqui a lógica de roteamento completa com as 5 prioridades que definimos antes)
        # ...
        # Exemplo da estrutura (COLE SUA LÓGICA DE ROTEAMENTO COMPLETA AQUI):
        # PRIORIDADE 1: Resposta a uma oferta de redirecionamento...
        if not agente_escolhido and pending_redirection_target_agent_name and pergunta_lower in affirmatives_for_redirection:
            # ... (lógica para escolher agente e preparar pergunta_para_agente)
            if pending_redirection_target_agent_name == simulador.name: agente_escolhido = simulador
            elif pending_redirection_target_agent_name == consultor.name: agente_escolhido = consultor
            elif pending_redirection_target_agent_name == suporte.name: agente_escolhido = suporte
            if agente_escolhido:
                print(f"🤖 Ok! Direcionando para o {agente_escolhido.name} para prosseguir...")
                if last_user_question: pergunta_para_agente = f"O usuário confirmou que quer prosseguir com a sugestão. A discussão anterior originou-se da pergunta: '{last_user_question}'. Por favor, continue."
                else: pergunta_para_agente = f"O usuário confirmou que gostaria de falar com você. Por favor, pergunte como pode ajudar para '{pergunta_original_usuario}'."
            pending_redirection_target_agent_name = None

        # PRIORIDADE 2: Continuar diálogo com SimuladorFinanceiro...
        if not agente_escolhido and last_agent_used_name == simulador.name and last_agent_response_summary and \
           "ACAO_SIMULAR:" not in last_agent_response_summary and \
           "simulação realizada" not in last_agent_response_summary.lower() and \
           "parâmetros inválidos" not in last_agent_response_summary.lower() and \
           not (pergunta_lower in frases_de_encerramento or pergunta_lower in saudacoes_comuns or \
                any(kw in pergunta_lower for kw in keywords_pedir_consultor + keywords_pedir_suporte)):
            agente_escolhido = simulador
            if last_agent_response_summary:
                pergunta_para_agente = f"Contexto: Sua última pergunta para o usuário foi: '{last_agent_response_summary}'. O usuário agora respondeu: '{pergunta_original_usuario}'. Por favor, continue a coleta de dados ou gere ACAO_SIMULAR."

        # PRIORIDADE 3: Pedido explícito de agente...
        if not agente_escolhido:
            # ... (lógica de pedido explícito como antes) ...
            if any(kw in pergunta_lower for kw in keywords_pedir_consultor): agente_escolhido = consultor
            elif any(kw in pergunta_lower for kw in keywords_pedir_suporte): agente_escolhido = suporte
            elif any(kw in pergunta_lower for kw in keywords_pedir_simulador): agente_escolhido = simulador
            elif last_agent_response_summary and any(kw in pergunta_lower for kw in ["falar com ele", "quero ele", "ele pode", "falar com ela", "quero ela", "ela pode"]):
                if consultor.name.lower() in last_agent_response_summary.lower(): agente_escolhido = consultor
                elif suporte.name.lower() in last_agent_response_summary.lower(): agente_escolhido = suporte
                elif simulador.name.lower() in last_agent_response_summary.lower(): agente_escolhido = simulador
            if agente_escolhido:
                 print(f"🤖 Direcionando para o {agente_escolhido.name} conforme solicitado ou inferido.")
                 if last_user_question and last_agent_response_summary and last_agent_used_name and last_agent_used_name != agente_escolhido.name:
                    context_preamble = f"Contexto: O usuário estava conversando com o {last_agent_used_name} sobre '{last_user_question}', que respondeu (resumo): '{last_agent_response_summary}'. "
                    pergunta_para_agente = f"{context_preamble}Agora o usuário pede para falar com você sobre: '{pergunta_original_usuario}'"

        # PRIORIDADE 4: Follow-up para o último agente usado...
        is_direct_follow_up = any(kw in pergunta_lower for kw in keywords_follow_up_generico + keywords_consultor_follow_up)
        if not agente_escolhido and last_agent_used_name and is_direct_follow_up and \
           not (last_agent_used_name == simulador.name and last_agent_response_summary and "ACAO_SIMULAR:" not in last_agent_response_summary and "simulação realizada" not in last_agent_response_summary.lower() and "parâmetros inválidos" not in last_agent_response_summary.lower() ) :
            if last_agent_used_name == consultor.name: agente_escolhido = consultor
            elif last_agent_used_name == suporte.name: agente_escolhido = suporte
            elif last_agent_used_name == simulador.name : agente_escolhido = simulador
            if agente_escolhido: print(f"🤖 Entendido, continuando com nosso {agente_escolhido.name} sobre o assunto anterior...")

        # PRIORIDADE 5: Roteamento por scores...
        if not agente_escolhido:
            # ... (lógica de scores e fallback como antes) ...
            score_consultor = sum(3 for kw in keywords_consultor_forte if kw in pergunta_lower) + sum(1 for kw in keywords_consultor_geral if kw in pergunta_lower)
            score_suporte = sum(3 for kw in keywords_suporte_forte if kw in pergunta_lower) + sum(1 for kw in keywords_suporte_geral if kw in pergunta_lower)
            score_simulador = sum(3 for kw in keywords_simulador_forte if kw in pergunta_lower) + sum(1 for kw in keywords_simulador_geral if kw in pergunta_lower)
            agentes_com_score = []
            if score_consultor > 0: agentes_com_score.append((score_consultor, consultor))
            if score_suporte > 0: agentes_com_score.append((score_suporte, suporte))
            if score_simulador > 0: agentes_com_score.append((score_simulador, simulador))
            agentes_com_score.sort(key=lambda x: x[0], reverse=True)
            if agentes_com_score:
                top_score = agentes_com_score[0][0]
                top_scorers_agents = [agent_obj for score, agent_obj in agentes_com_score if score == top_score]
                if len(top_scorers_agents) == 1: agente_escolhido = top_scorers_agents[0]
                else:
                    if simulador in top_scorers_agents and any(kw in pergunta_lower for kw in keywords_simulador_forte + keywords_simulador_geral): agente_escolhido = simulador
                    elif suporte in top_scorers_agents and any(kw in pergunta_lower for kw in keywords_suporte_forte + keywords_suporte_geral): agente_escolhido = suporte
                    else: agente_escolhido = consultor
                    print(f"🤖 Sua pergunta parece ter múltiplos aspectos. Direcionando para {agente_escolhido.name}.")
            else:
                if any(term in pergunta_lower for term in ["simula", "simular", "projeção", "calcular rendimento", "quanto rende", "simulacao", "simulador"]): agente_escolhido = simulador
                elif any(term in pergunta_lower for term in ["problema", "erro", "não funciona", "nao funciona", "dificuldade", "falha", "ajuda com app", "ajuda com site", "deu pau", "quebrou", "internet", "login", "senha", "acesso", "excluir", "deletar", "cancelar conta"]): agente_escolhido = suporte
                elif any(term in pergunta_lower for term in ["dívida", "divida", "dívidas", "dividas", "endividado", "como sair das dívidas", "sair da divida", "economizar", "orçamento", "aprender", "entender", "dica", "educa", "conceito", "significa", "o que é", "como funciona", "lição", "explicar"]): agente_escolhido = consultor
                else: agente_escolhido = consultor
                print(f"🤖 Encaminhando para {agente_escolhido.name} para melhor assistência.")

        # Preparar pergunta com contexto (lógica mantida)
        if agente_escolhido and last_agent_used_name and agente_escolhido.name != last_agent_used_name and \
           not (last_agent_used_name == simulador.name and agente_escolhido == simulador and "ACAO_SIMULAR:" not in last_agent_response_summary and "simulação realizada" not in last_agent_response_summary.lower() and "parâmetros inválidos" not in last_agent_response_summary.lower()) and \
           not (agente_escolhido == simulador and pergunta_original_usuario.lower() in affirmatives_for_redirection and pending_redirection_target_agent_name == simulador.name):
            if last_user_question:
                context_preamble = f"Contexto: O usuário estava conversando com o {last_agent_used_name} sobre '{last_user_question}'. "
                if last_agent_response_summary and last_agent_response_summary != "Não houve resposta anterior para resumir.": context_preamble += f"O {last_agent_used_name} respondeu (resumo): '{last_agent_response_summary}'. "
                is_generic_switch_or_follow_up = (any(kw in pergunta_original_usuario.lower() for kw in keywords_pedir_consultor + keywords_pedir_suporte + keywords_pedir_simulador) or is_direct_follow_up)
                if is_generic_switch_or_follow_up: pergunta_para_agente = f"{context_preamble}O usuário gostaria de continuar o assunto ou ter sua perspectiva sobre o tópico: '{last_user_question}' Por favor, prossiga sobre '{last_user_question}'."
                else: pergunta_para_agente = f"{context_preamble}Agora, o usuário pergunta sobre um novo aspecto ou relacionado: '{pergunta_original_usuario}'"
                print(f"🤖 Mudando para o {agente_escolhido.name} com contexto da conversa anterior...")

        # Definir session_id
        if agente_escolhido == consultor: session_id_para_agente_atual = session_id_consultor
        elif agente_escolhido == suporte: session_id_para_agente_atual = session_id_suporte
        elif agente_escolhido == simulador: session_id_para_agente_atual = session_id_simulador
        else:
            agente_escolhido = consultor; session_id_para_agente_atual = session_id_consultor
            pergunta_para_agente = pergunta_original_usuario
            print(f"🤖 Ops! Tive um problema ao direcionar, mas o {consultor.name} vai te ajudar.")

        resposta_do_agente = ""
        if agente_escolhido:
            print(f"🔄 Consultando o {agente_escolhido.name}...")
            resposta_do_agente = call_agent(agente_escolhido, pergunta_para_agente, session_service, user_id_conversa, session_id_para_agente_atual)

            acao_simular_tag = "ACAO_SIMULAR:"
            tag_index = resposta_do_agente.find(acao_simular_tag)

            if tag_index != -1 and agente_escolhido == simulador:
                params_part_str = resposta_do_agente[tag_index + len(acao_simular_tag):]
                match = re.search(r"\{\{(.*?)\}\}", params_part_str, re.DOTALL)
                if match:
                    json_like_content = match.group(1); processed_params_str = f"{{{json_like_content}}}"
                    processed_params_str = processed_params_str.replace("None", "null").replace("True", "true").replace("False", "false").replace("'", "\"")
                    try:
                        params = json.loads(processed_params_str)
                        valor_inicial_sim = float(params.get('valor_inicial', 0)); aporte_mensal_sim = float(params.get('aporte_mensal', 0))
                        taxa_juros_anual_nominal_sim = float(params.get('taxa_juros_anual_nominal', 0)); periodo_anos_sim = float(params.get('periodo_anos', 0))
                        taxa_inflacao_anual_sim = float(params.get('taxa_inflacao_anual_esperada', 0.0))
                        if periodo_anos_sim > 0 and taxa_juros_anual_nominal_sim >= 0 and taxa_inflacao_anual_sim >=0:
                            display(Markdown(f"### 📊 **Preparando sua Simulação de Investimento (Valores Corrigidos pela Inflação)**"))
                            display(Markdown(f"- Valor Inicial: **R$ {valor_inicial_sim:,.2f}**")); display(Markdown(f"- Aporte Mensal: **R$ {aporte_mensal_sim:,.2f}**"))
                            display(Markdown(f"- Taxa de Juros Nominal Anual: **{taxa_juros_anual_nominal_sim*100:.2f}%**")); display(Markdown(f"- Período: **{periodo_anos_sim:.2f} anos** ({periodo_anos_sim*12:.1f} meses)"))
                            display(Markdown(f"- Inflação Anual Esperada: **{taxa_inflacao_anual_sim*100:.2f}%**"))
                            resultados_simulacao = simular_investimento_juros_compostos(valor_inicial_sim, aporte_mensal_sim, taxa_juros_anual_nominal_sim, periodo_anos_sim, taxa_inflacao_anual_sim)
                            display(Markdown("--- \n### **Resultados da Simulação (Poder de Compra de Hoje):**"))
                            display(Markdown(f"O valor final do seu investimento, já descontada a inflação esperada, seria de aproximadamente: **R$ {resultados_simulacao['valor_final_real']:,.2f}**."))
                            display(Markdown(f"Para alcançar isso, o total nominal investido (inicial + aportes) seria de: R$ {resultados_simulacao['total_investido_nominal']:,.2f}."))
                            display(Markdown("---"))
                            # Remoção do download do arquivo
                            print("\n🤖 Gostaria de fazer outra simulação ou perguntar sobre outro assunto?")
                            last_simulation_results_context = {'valor_final_real': resultados_simulacao['valor_final_real'],'total_investido_nominal': resultados_simulacao['total_investido_nominal']}
                            last_agent_response_summary = f"Simulação (corrigida pela inflação) realizada."
                        else:
                            print("🤖 Para simular, preciso de taxa de juros (>=0) e período (>0) válidos."); last_agent_response_summary = "Tentativa de simulação com parâmetros inválidos."; last_simulation_results_context = None
                    except Exception as e: print(f"🤖 Problema ao processar simulação: {e}. JSON: '{processed_params_str}'"); last_agent_response_summary = "Erro ao processar simulação."; last_simulation_results_context = None
                else: print(f"🤖 ERRO: Formato ACAO_SIMULAR não reconhecido. Resposta: '{resposta_do_agente}'"); last_agent_response_summary = resposta_do_agente[:100]; last_simulation_results_context = None
                resposta_do_agente = ""

            elif agente_escolhido == consultor and "simuladorfinanceiro" in resposta_do_agente.lower() and \
                 ("gostaria de prosseguir" in resposta_do_agente.lower() or "posso te direcionar" in resposta_do_agente.lower()):
                print(f"\n🤖 Resposta:\n{resposta_do_agente}")
                pending_redirection_target_agent_name = simulador.name
                last_agent_response_summary = resposta_do_agente.split('.')[0] + "." if '.' in resposta_do_agente else resposta_do_agente[:100]
            else:
                if resposta_do_agente: print(f"\n🤖 Resposta:\n{resposta_do_agente}"); last_agent_response_summary = resposta_do_agente.split('.')[0] + "." if '.' in resposta_do_agente else resposta_do_agente[:100]
                else: last_agent_response_summary = "O agente não forneceu uma resposta clara ou a ação já foi exibida."

            last_user_question = pergunta_original_usuario
            last_agent_used_name = agente_escolhido.name
        else:
            print("\n🤖 Resposta:\nDesculpe, ocorreu um erro ao processar sua solicitação.")
            last_agent_used_name = None; last_user_question = ""; last_agent_response_summary = ""; pending_redirection_target_agent_name = None; last_simulation_results_context = None

        if agente_escolhido == consultor and isinstance(resposta_do_agente, str) and resposta_do_agente.strip() and \
           not (resposta_do_agente.find(acao_simular_tag) != -1 ) and "Lembre-se que estas são informações gerais" not in resposta_do_agente:
             print(f"\nLembre-se: As informações fornecidas são gerais e educativas...")
        print("-" * 50)

if __name__ == "__main__":
    chatbot_financeiro()

🏦 Bem-vindo ao Chatbot Financeiro Completo! 🏦
Sou seu assistente virtual para o mundo financeiro. Como posso te ajudar hoje?
Posso te ajudar com nosso ConsultorFinanceiro (para informações e educação financeira), nosso SimuladorFinanceiro (para simulações de investimento), ou nosso SuporteOperacional (para processos e suporte técnico).

IMPORTANTE: Eu forneço informações gerais e educativas, e não aconselhamento financeiro personalizado ou recomendações específicas.
As condições de mercado podem mudar rapidamente. Para decisões financeiras importantes, especialmente investimentos, consulte um profissional qualificado. (Data de referência: 17/05/2025)

❓ Digite sua pergunta: sair
🤖 Entendido. Espero ter ajudado! Se tiver mais dúvidas no futuro, é só chamar. Até a próxima!
